In [ ]:
# Defining Libraries:
# Using all of the channels
import mne
import numpy as np
from scipy.signal import welch, stft
from scipy.stats import skew, kurtosis, entropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
file_participant_1 = 'Data/RawData/Participant_1.edf'
file_participant_2 = 'Data/RawData/Participant_2.edf'
file_participant_3 = 'Data/RawData/Participant_3.edf'
file_participant_4 = 'Data/RawData/Participant_4.edf'
file_participant_5 = 'Data/RawData/Participant_5.edf'
file_participant_6 = 'Data/RawData/Participant_6.edf'
file_participant_7 = 'Data/RawData/Participant_7.edf'
file_participant_8 = 'Data/RawData/Participant_8.edf'
file_participant_9 = 'Data/RawData/Participant_9.edf'
file_participant_10 = 'Data/RawData/Participant_10.edf'
file_participant_11 = 'Data/RawData/Participant_11.edf'
file_participant_12 = 'Data/RawData/Participant_12.edf'
file_participant_13 = 'Data/RawData/Participant_13.edf'

edf_data_files = [
    file_participant_1,
    file_participant_2,
    file_participant_3,
    file_participant_4,
    file_participant_5,
    file_participant_6,
    file_participant_7,
    file_participant_8,
    file_participant_9,
    file_participant_10,
    file_participant_11,
    file_participant_12,
    file_participant_13
]

In [ ]:
# Defining Channels of Interest:

#channels_of_interest = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

In [ ]:
# Remove start and finish and remove the breaks as well:

# Loading the first file to use as a reference for channel names
print("Loading reference EDF file...")
reference_raw = mne.io.read_raw_edf(edf_data_files[0], preload=True)
reference_channels = reference_raw.info['ch_names']

raw_objects = []

# Define the segments of interest in seconds:
segments = [
    (30, 90),  # "I"
    (120, 180),  # "Yes"
    (210, 270),  # "No"
    (300, 360),  # "Want"
    (390, 450),  # "Help"
    (480, 540),  # "More"
    (570, 630),  # "That"
    (660, 720),  # "Stop"
    (750, 810),  # "Open"
    (840, 900)   # "Close"
]

for file_path in edf_data_files:
    print(f"Loading and cropping EDF file: {file_path}...")
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.pick_channels(reference_channels)
    
    # Create an empty list to store the segments:
    data_segments = []
    
    for start, end in segments:
        segment = raw.copy().crop(tmin=start, tmax=end)
        data_segments.append(segment)
    
    # Concatenate the segments:
    raw_concatenated = mne.concatenate_raws(data_segments)
    raw_objects.append(raw_concatenated)

In [ ]:
# Displaying the resulting files after segmentation:

raw_objects

In [ ]:
# Concatenating all loaded and processed objects:

print("Concatenating all processed EDF files...")
raw = mne.concatenate_raws(raw_objects)

In [ ]:
raw

In [ ]:
# Function to Preprocess Raw Data:

def preprocess_raw_data(raw):

    # Handling NaNs: Replace NaNs with the mean of the respective channel
    raw_data = raw.get_data()

    for i in range(raw_data.shape[0]):
        nan_indices = np.isnan(raw_data[i])

        if np.any(nan_indices):
            mean_value = np.nanmean(raw_data[i])
            raw_data[i, nan_indices] = mean_value

    raw._data = raw_data

    # Filtering: Bandpass filter between 0.5-30 Hz
    raw.filter(0.5, 30., fir_design='firwin')
    
    # Artifact Removal: ICA
    ica = mne.preprocessing.ICA(n_components = 14, random_state = 97, max_iter = 800)
    ica.fit(raw)
    raw = ica.apply(raw)
    
    # Spatial Filtering: Common Average Reference (CAR)
    raw.set_eeg_reference('average', projection = True)
    
    # Channel Interpolation: Interpolate bad channels
    raw.interpolate_bads()

    # Baseline Correction: Apply baseline correction using the mean of the segment
    raw.apply_function(lambda x: x - np.mean(x), picks = 'eeg')
    
    return raw

In [ ]:
# Preprocessing the Raw Data:

raw = preprocess_raw_data(raw)

In [ ]:
raw

In [ ]:
# Creating Fixed-Length Epochs:

epoch_duration = 60  # seconds
start_times = np.arange(0, raw.times[-1] - epoch_duration, epoch_duration)
end_times = start_times + epoch_duration

In [ ]:
print("Start Times: ", start_times)
print("\nTotal Number of Start Times: ", len(start_times))

In [ ]:
print("End Times: ", end_times)
print("\nTotal Number of End Times: ", len(end_times))

In [ ]:
# Defining the Words:

words = ['I', 'Yes', 'No', 'Want', 'Help', 'More', 'That', 'Stop', 'Open', 'Close']

In [ ]:
# Defining the Frequency:

sfreq = raw.info['sfreq']
print("Frequency Across Channels: ", sfreq, "Hz")

In [ ]:
# Feature Extraction Function:

def extract_features(epoch_data, sfreq):

    # Calculating Statistical Features:
    mean_vals = np.mean(epoch_data, axis = 1) # Mean value of the signal for each channel.
    std_vals = np.std(epoch_data, axis = 1) # Standard deviation of the signal for each channel.
    skew_vals = skew(epoch_data, axis = 1) # Skewness of the signal for each channel, indicating asymmetry.
    kurt_vals = kurtosis(epoch_data, axis = 1) # Kurtosis of the signal for each channel, indicating peakedness.

    # Power Spectral Density (PSD) Features:
    freqs, psd = welch(epoch_data, sfreq, nperseg = int(sfreq)) # Computes the PSD using Welch’s method.
    # Average power in the theta (4-8 Hz), alpha (8-12 Hz), and beta (12-30 Hz) frequency bands:
    theta_power = psd[:, (freqs > 4) & (freqs <= 8)].mean(axis = 1)
    alpha_power = psd[:, (freqs > 8) & (freqs <= 12)].mean(axis = 1)
    beta_power = psd[:, (freqs > 12) & (freqs <= 30)].mean(axis = 1)

    # Short-Time Fourier Transform (STFT) Features:
    _, _, Zxx = stft(epoch_data, fs = sfreq, nperseg = int(sfreq/2)) # Computes the STFT, which provides time-frequency representation of the signal.
    stft_power = np.abs(Zxx).mean(axis = 2) # Mean power from the STFT representation, averaged over time.
    
    # Entropy Feature:
    entropy_vals = np.array([entropy(np.abs(epoch_data[channel, :])) for channel in range(epoch_data.shape[0])]) # Computes the entropy of the signal for each channel, indicating the complexity or randomness of the signal.

    # Combining Features:
    features = np.stack([
        mean_vals,
        std_vals,
        skew_vals,
        kurt_vals,
        theta_power,
        alpha_power,
        beta_power,
        stft_power.mean(axis=1),
        entropy_vals
    ], axis = 1) # Combines all extracted features into a single array with each feature as a column.

    return features

In [ ]:
# Segmenting the Data into Epochs and Sub-Epochs:

print("Segmenting data into 60-second epochs, then into 2-second sub-epochs, and extracting features...")

labeled_features_data = []
sub_epoch_duration = 2  # seconds

for i, (start, end) in enumerate(zip(start_times, end_times)): # Loops through each 30-second epoch:
    start_sample = int(start * sfreq)
    end_sample = int(end * sfreq)
    epoch_data, _ = raw[:, start_sample:end_sample] # Extracts the EEG data for the current epoch.
    word_label = words[i % len(words)] # Assigns a label to the current epoch using a list of predefined words.
    
    for j in range(int(epoch_duration / sub_epoch_duration)): # Iterate over Sub-Epochs:
        sub_start = j * sub_epoch_duration * int(sfreq)
        sub_end = (j + 1) * sub_epoch_duration * int(sfreq)
        sub_epoch_data = epoch_data[:, sub_start:sub_end]
        # Calls the extract_features function to extract from sub-epoch:
        features = extract_features(sub_epoch_data, sfreq)
        labeled_features_data.append((features, word_label)) # Stores the extracted features along with the label.

In [ ]:
print(labeled_features_data)

In [ ]:
len(labeled_features_data)

In [ ]:
# Extracting Features and Labels:

#features, labels = zip(*labeled_features_data) if labeled_features_data else ([], [])
#features = np.array(features) if features else np.empty((0, 0))
#labels = np.array(labels)

features = np.array([f[0] for f in labeled_features_data])
labels = np.array([f[1] for f in labeled_features_data])

In [ ]:
len(features)

In [ ]:
len(labels)

In [ ]:
# Flattening the last two dimensions of the features array.
# To transform the 3D feature array into a 2D array where each row represents a single sample and each column represents a feature.
features_2d = features.reshape(features.shape[0], -1)

# Handling NaN values: replacing them with the column mean
imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
features_imputed = imputer.fit_transform(features_2d)

# Scaling the features
# To standardize the features by scaling them so that they have a mean of 0 and a standard deviation of 1.
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_imputed)

In [ ]:
# PCA
pca = PCA(n_components = 0.95) 
features_pca = pca.fit_transform(features_scaled)

# Mutual Information
num_sub_epochs_per_epoch = int(epoch_duration / sub_epoch_duration)
total_sub_epochs = num_sub_epochs_per_epoch * len(start_times)

num_features = features_scaled.shape[1]
k_best = min(num_features, 20)  # Ensure k does not exceed the number of available features
mi_selector = SelectKBest(mutual_info_classif, k=k_best)
features_mi = mi_selector.fit_transform(features_scaled, labels)

# Choosing feature set to use for further model training
selected_features = features_mi

num_features_mi = features_mi.shape[1]  # Number of features after MI

In [ ]:
num_features_mi

In [ ]:
num_features

In [ ]:
k_best

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(
    selected_features, labels, test_size=0.3, random_state=42, stratify=labels)

val_features, test_features, val_labels, test_labels = train_test_split(
    test_features, test_labels, test_size=0.5, random_state=42, stratify=test_labels)

print(f"Training data size: {len(train_features)}")
print(f"Validation data size: {len(val_features)}")
print(f"Testing data size: {len(test_features)}")

# Counting occurrences of each label in the training, validation, and testing sets
train_label_counts = Counter(train_labels)
val_label_counts = Counter(val_labels)
test_label_counts = Counter(test_labels)

# Calculating the total number of samples in each set
total_train = len(train_labels)
total_val = len(val_labels)
total_test = len(test_labels)

# Printing the distribution of each label in each set
print("Training set label distribution:")
for label, count in train_label_counts.items():
    print(f"{label}: {count} ({count / total_train * 100:.2f}%)")

print("\nValidation set label distribution:")
for label, count in val_label_counts.items():
    print(f"{label}: {count} ({count / total_val * 100:.2f}%)")

print("\nTesting set label distribution:")
for label, count in test_label_counts.items():
    print(f"{label}: {count} ({count / total_test * 100:.2f}%)")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class EEGTransformerEncoder(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=256, nhead=8, num_layers=16, dim_feedforward=512, dropout_rate=0.1, noise_std=0.01):
        super(EEGTransformerEncoder, self).__init__()
        self.noise_std = noise_std
        self.linear_in = nn.Linear(input_dim, d_model)
        self.dropout_in = nn.Dropout(dropout_rate)
        
        # Using pre-LayerNorm
        encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                                dim_feedforward=dim_feedforward, 
                                                dropout=dropout_rate, 
                                                activation='gelu', 
                                                norm_first=True)

        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.batch_norm = nn.BatchNorm1d(d_model)
        self.linear_out = nn.Linear(d_model, num_classes)
        self.dropout_out = nn.Dropout(dropout_rate)

    def forward(self, x):
        if self.training and self.noise_std > 0.0:
            noise = torch.randn_like(x) * self.noise_std
            x = x + noise

        x = self.linear_in(x)
        x = self.dropout_in(x)
        
        x = self.transformer_encoder(x)  # Transformer encoder with pre-LayerNorm

        x = self.batch_norm(x)
        x = self.dropout_out(x)
        x = self.linear_out(x)

        return F.log_softmax(x, dim=1)

model = EEGTransformerEncoder(input_dim=num_features_mi, num_classes=10)
print(model)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

class EEGDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
# Encoding string labels to integers
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

print(test_labels)
print(test_labels_encoded)
    
train_dataset = EEGDataset(train_features, train_labels_encoded)
val_dataset = EEGDataset(val_features, val_labels_encoded)
test_dataset = EEGDataset(test_features, test_labels_encoded)

# Defining DataLoader
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [31]:
import torch
import torch.nn.functional as F

# Training parameters
learning_rate = 1e-5
epochs = 600

# Using CrossEntropyLoss for classification
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-8)

train_accuracies = []
val_accuracies = []

l1_lambda = 0.0001

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for features, labels in train_loader:
        optimizer.zero_grad()
        output = model(features)
        loss = criterion(output, labels)
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += l1_lambda * l1_norm  # L1 regularization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        train_loss += loss.item()
        predictions = torch.max(output, 1)[1]
        train_correct += (predictions == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = train_correct / total_train
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for features, labels in val_loader:
            output = model(features)
            loss = criterion(output, labels)
            val_loss += loss.item()
            predictions = torch.max(output, 1)[1]
            val_correct += (predictions == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = val_correct / total_val
    val_accuracies.append(val_accuracy)

    print(f'Epoch {epoch+1}, Loss: {train_loss / total_train}, Training Accuracy: {train_accuracy}, '
          f'Validation Loss: {val_loss / total_val}, Validation Accuracy: {val_accuracy}')

Epoch 235, Loss: 0.08224101852584671, Training Accuracy: 0.8600732600732601, Validation Loss: 0.0016875428521735036, Validation Accuracy: 0.9111111111111111
